# Number Theory

## Integer Square Root

In [ ]:
from theonum import isqrt

In [ ]:
isqrt(31)

In [ ]:
isqrt(144)

In [ ]:
isqrt(55**2+3)

In [ ]:
n = 99987513635933739420478972158060509974719355801**2+42378687
n

In [ ]:
isqrt(n, trace=True)

## Continued Fraction

In [ ]:
from theonum import cf_expansion, convergents, shcf
from sympy import Rational

In [ ]:
shcf(811,972)

In [ ]:
bs = cf_expansion(811,972)

bs

In [ ]:
[Rational(n,d) for n, d in convergents(bs)]

## Fermat

In [ ]:
def fermat(N, limit):
    squaresMod20 = {0,1,4,5,9,16}
    from itertools import count

    rr = 0
    start = isqrt(N)[0]+1

    for x in count(start):
        if x-start+1 > limit:
            print('NOT FOUND')
            break

        b = x**2 - N
        if b % 20 not in squaresMod20:
            continue
        (r,D) = isqrt(b)
        rr += 1
        if D==0:
            print(f"{N} = {N//(x-r)} x {N//(x+r)} = ({x} + {r}) x ({x} - {r})  {'OK' if N==(x+r)*(x-r) else 'FAIL'}")
            print(f"start={start}, steps={x-start+1}/{rr}")
            #print(factorint(b))
            break

In [ ]:
N = 2041

fermat(N, 100)

In [ ]:
from sympy import prime

N = prime(50) * prime(150)

fermat(N, 110)

In [ ]:
N = prime(500) * prime(1500)

fermat(N, 10000)

## Sqrt 1

In [ ]:
def find_sqrt_1(N, limit):
    from itertools import count
    from math import gcd

    print('steps x  Q')

    start = isqrt(N)[0]+1
    for x in count(start):
        if x-start+1 > limit:
            print('NOT FOUND')
            break

        Q = x**2 % N
        if x-start+1 < 10: print(f'{x-start+1} {x} {Q}')
        if Q == 1:
            break
    if Q==1:
        print(f'BINGO {x-start+1} steps,  x={x}  {(x+1)*(x-1)//N}N  F={gcd(x-1,N)}')

In [ ]:
N = 2041

find_sqrt_1(N, 400)

In [ ]:
N = prime(50) * prime(150)

find_sqrt_1(N, 20000)

In [ ]:
N = prime(1000) * prime(1010)

find_sqrt_1(N, 4000000)

## Pollard $\rho$

In [ ]:
from math import gcd

def pollarho_demo(p,q,a,maxiter=100):
    n = p*q
    b = a
    print(f'{p} x {q} = {n}')
    print(f'{"tortoise":>10} {"hare":>34} {"factor":>34}')
    for steps in range(maxiter):
        d = gcd(a-b,n)
        print(f'{a:10} {a%p:10} {a%q:10} | {b:10} {b%p:10} {b%q:10} | {d:10}')
        if d != 1 and d != n: print(a,b,steps); break
        a = (a**2 + 1)%n
        b = (((b**2 + 1)%n)**2 + 1)%n

def pollarho(n,a,maxiter=100):
    b = a
    for steps in range(maxiter):
        d = gcd(a-b,n)
        if steps <10: print(f'{a:10}  |  {b:10}  | {d:10}')
        if d != 1 and d != n: break
        if steps==10: print('...')
        a = (a**2 + 1)%n
        b = (((b**2 + 1)%n)**2 + 1)%n
    p = d
    q = n//p
    print(f'{n}={p}x{q}  {n==p*q} {steps} steps')

In [ ]:
pollarho_demo(11,13,14)

In [ ]:
pollarho_demo(prime(50),prime(70), 7)

In [ ]:
pollarho(prime(1000)*prime(1250),333)

In [ ]:
pollarho(prime(10000)*prime(20000),333,maxiter=1000)

## Quadratic Sieve

https://en.wikipedia.org/wiki/Quadratic_sieve

In [ ]:
from itertools import count, islice
from sympy import prime, factorint
from theonum import isqrt
import numpy as np
from math import gcd

def prepareSieve(n, p, check=False):

    def pred(x):
        # quadratic residue not checked at the moment
        #https://crypto.stackexchange.com/questions/35615/in-the-quadratic-sieve-why-restrict-the-factor-base
        if not check: return True
        return True

    smallprimes = islice(filter(pred, map(prime, count(1))),p)
    start = isqrt(n)[0]+1
    return start, list(smallprimes)

def issmooth(n,someprimes):
    x = n
    fact = {}
    for p in someprimes:
        fact[p] = 0
        q,r = divmod(x,p)
        while r == 0:
            x = q
            fact[p] += 1
            q,r = divmod(x,p)
    return x,fact

def mkvec(fact, B, at, binary=False):
    x = [' ' for _ in B]
    for p,v in fact.items():
        x[at[p]] = str(v%2) if binary else str(v)
    return ''.join(x).replace('0','·')

def mkMatrix(B,at):
    r = []
    for x,_,fact in smooth:
        r.append([x=='1' for x in mkvec(fact,B,at,True)])
    return np.array(r).astype(int)

def reduce(M):
    I = np.eye(M.shape[0]).astype(int)
    n,m = M.shape
    p = 0
    for c in range(m):
        #print(c)
        r = np.where(M[p:,c]==1)[0]
        if len(r)==0:
            continue
        r = r[0]+p
        M[[p,r]] = M[[r,p]]
        I[[p,r]] = I[[r,p]]
        for k in range(p+1,n):
            if M[k,c] == 1:
                M[k] = (M[k] + M[p])%2
                I[k] = (I[k] + I[p])%2
        p += 1
        #print(M)
    #print(p)
    return I[:p], I[p:]

def tryq(js, smooth):
    print('x: ',[smooth[j][0] for j in js])
    print('Q: ',[smooth[j][1] for j in js])
    u = 1
    v = 1
    for j in js:
        u *= smooth[j][0]
        v *= smooth[j][1]

    if isqrt(v)[1] != 0:
        print('BAD SUBSET')
        return
    print(f'(Πx)^2  ΠQ', u**2%N, v%N)
    r = isqrt(v)[0]
    print('Πx  √ΠQ', u,r)
    print('Πx  √ΠQ mod N', u%N,r%N)
    fact = gcd(abs(u-r),N)
    print('factor:', fact, fact!=N and fact!=1)

def findSmooth(limit, start, smallprimes):
    print('   steps        x        Q         factors            mod 2')

    at = {p:k for k,p in enumerate(smallprimes)}
    smooth = []

    for x in count(start):
        if len(smooth) == limit:
            break
        Q = x**2 % N
        y, fact = issmooth(Q,smallprimes)
        if y==1:
            smooth += [(x,Q,fact)]
        else:
            continue
        (r,D) = isqrt(Q)
        if D==0:
            sx = f'{x:4}'
            if x%N != r%N and x%N != (-r)%N:
                factor = gcd(abs(x-r),N)
                sx += f' OK {factor}'
        else:
            sx = '    '
        print(f'{x-start:8} {x:8} {Q:8} {mkvec(fact,smallprimes,at):>15}  {mkvec(fact,smallprimes,at,binary=True):>15}  {sx}')
        if D==0:
            break

    r = []
    for x,_,fact in smooth:
        r.append([x=='1' for x in mkvec(fact,smallprimes,at,True)])
    M = np.array(r).astype(int)
    return smooth, M

def makeFactor(smooth, M):
    T = M.copy()
    Span,Null = reduce(T)
    #print(Span)
    print(Null)
    #print(M.shape, Null.shape)
    print((Null @ M) % 2)
    for sol in Null:
        print('----------------')
        print(M[np.where(sol==1)[0]])
        tryq(np.where(sol==1)[0], smooth)

In [ ]:
N = 2041

st, smpr = prepareSieve(N,5)

print(f'start = ceil sqrt({N}) = {st}, small primes: {smpr}')

sm, mt = findSmooth(len(smpr)+1, st, smpr)

makeFactor(sm,mt)

In [ ]:
N = prime(100)*prime(103)

st, smpr = prepareSieve(N,5)

print(f'start = ceil sqrt({N}) = {st}, small primes: {smpr}')

sm, mt = findSmooth(len(smpr)+1, st, smpr)

makeFactor(sm,mt)

In [ ]:
N = prime(100)*prime(150)

st, smpr = prepareSieve(N,5)

print(f'start = ceil sqrt({N}) = {st}, small primes: {smpr}')

sm, mt = findSmooth(len(smpr)+1, st, smpr)

makeFactor(sm,mt)

In [ ]:
N = prime(500)*prime(1500)

st, smpr = prepareSieve(N,5)

print(f'start = ceil sqrt({N}) = {st}, small primes: {smpr}')

sm, mt = findSmooth(len(smpr)+1, st, smpr)

makeFactor(sm,mt)

In [ ]:
N = prime(5000)*prime(15000)

st, smpr = prepareSieve(N,6)

print(f'start = ceil sqrt({N}) = {st}, small primes: {smpr}')

sm, mt = findSmooth(len(smpr)+1, st, smpr)

makeFactor(sm,mt)

In [ ]:
N = 9788111

st, smpr = prepareSieve(N,5)

print(f'start = ceil sqrt({N}) = {st}, small primes: {smpr}')

sm, mt = findSmooth(len(smpr)+1, st, smpr)

makeFactor(sm,mt)